In [1]:
import pandas as pd
import numpy as np
import sys, zipfile, os, time
import matplotlib.pyplot as plt
import glob
import datetime
import urllib3
import cdsapi
import xarray
import shutil
import warnings
import pickle
import h5py
import netCDF4
warnings.filterwarnings('ignore')

sys.path.append(os.getcwd()+'/../cds-backend/code/')
import cds_eua3 as eua

In [2]:
def get_adj(station):
    import datetime
    """Compare created adjustment to downloaded adjustment
    """
    date0 = datetime.datetime.strptime('1900-01-01', "%Y-%m-%d")

    file = glob.glob('./Humidity_adjustments/files/downloaded/downloaded_'+station+'/dest_*_relative_humidity_adjusted.nc')[0]
    data = eua.CDMDataset(file).to_dataframe()
    daydata = data[data.time.dt.hour > 6][data.time.dt.hour <= 18]
    nightdata = data[data.time.dt.hour <= 6].append(data[data.time.dt.hour > 18])
    
    c = cdsapi.Client()
    r = c.retrieve('insitu-comprehensive-upper-air-observation-network',
                   {'variable': 'relative_humidity',
                    'optional':['humidity_bias_estimate'],
                    'statid': station,
                    'pressure_level':[10,20,30,50,70,100,150,200,250,300,400,500,700,850,925,1000]
                   }
                  )
    r.download(target='download.zip')
    assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
    z = zipfile.ZipFile('download.zip')
    z.extractall(path='./tocompare/hur/cds_'+station)
    z.close()
    cdsfile = glob.glob('./tocompare/hur/cds_'+ station +'/*.nc')
    cdsdata=eua.CDMDataset(cdsfile[0]).to_dataframe()
    daycdsdata = cdsdata[cdsdata.time.dt.hour > 6][cdsdata.time.dt.hour <= 18]
    nightcdsdata = cdsdata[cdsdata.time.dt.hour <= 6].append(cdsdata[cdsdata.time.dt.hour > 18])
    
    return daydata, nightdata, daycdsdata, nightcdsdata
#     breakdates.append(str(cdsdata.time.iloc[0]))#.year)+str(cdsdata.time.iloc[0].month)+str(cdsdata.time.iloc[0].day))
#     dates = breakdates
#     dates.reverse()
#     print(dates)
    
#     calc_day_out = {}
#     calc_night_out = {}
#     cds_day_out = {}
#     cds_night_out = {}
#     for i in [10,20,30,50,70,100,150,200,250,300,400,500,700,850,925,1000]:
#         calc_day_out[str(i)]=[]
#         calc_night_out[str(i)]=[]
#         cds_day_out[str(i)]=[]
#         cds_night_out[str(i)]=[]
#     for i in range(len(dates)-1):
#         for j in range(len(press)):
#             try:
#                 d_adjd = dayadj[j][i]
#                 calc_day_out[str(int(press[j]))].append(d_adjd)
#             except:
#                 calc_day_out[str(int(press[j]))].append(np.nan) 
#             try:
#                 d_cdsd = daydata[daydata.plev == press[j]*100][daydata.time >= dates[i]][daydata.time < dates[i+1]]
#                 cds_day_out[str(int(press[j]))].append(d_cdsd[var].iloc[0])
#             except:
#                 cds_day_out[str(int(press[j]))].append(np.nan)
#             try:
#                 n_cdsd = nightdata[nightdata.plev == press[j]*100][nightdata.time >= dates[i]][nightdata.time < dates[i+1]]
#                 cds_night_out[str(int(press[j]))].append(n_cdsd[var].iloc[0])
#             except:
#                 cds_night_out[str(int(press[j]))].append(np.nan)
#             try:
#                 n_adjd = nightadj[j][i]
#                 calc_night_out[str(int(press[j]))].append(n_adjd)
#             except:
#                 calc_night_out[str(int(press[j]))].append(np.nan)
                
#     return dates, cds_day_out, cds_night_out, calc_day_out, calc_night_out

In [3]:
def plt_adj(cdata, adata, plev, station, time, adj, save):
    cdata = cdata[cdata.plev == plev*100]
    adata = adata[adata.plev == plev*100]
    print(1)
    fig, ax = plt.subplots()
    ax.scatter(cdata['time'], cdata.humidity_bias_estimate, label='CDS')
    ax.scatter(adata['time'], adata.hur_bias_estimate, label='Calc')
    print(2)

    ax.set(xlabel='time', ylabel='Adjustment: '+adj,
           title= str(station)+' Temperature '+str(plev)+' hPa '+time)
    ax.legend()
    ax.grid()
    
    if save:
        fig.savefig(str(station)+'_Temperature_'+str(plev)+'_hPa_'+time+'.png')
    plt.show()
    return

In [4]:
station = '11035'
dd, nd, dcd, ncd = get_adj(station)

plt_adj(dcd, dd, 100, station, 'day', 'humidity_bias_estimate', save=False)


2021-05-03 14:58:06,853 INFO Welcome to the CDS
2021-05-03 14:58:06,854 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2021-05-03 14:58:07,911 INFO Request is completed
2021-05-03 14:58:07,912 INFO Downloading http://136.156.132.204/cache-compute-0001/cache/data2/adaptor.comprehensive_upper_air.retrieve-1620042926.6514504-17772-7-5da21943-5a67-49a2-98ed-875cb79a39f6.zip to download.zip (17.5M)
2021-05-03 14:58:08,677 INFO Download rate 22.9M/s  
2021-05-03 14:58:08,742 INFO Reading Profile on slice(None, None, None)


In [ ]:
plev = 500
cdata = dcd[dcd.plev == plev*100]
adata = dd[dd.plev == plev*100]
fig, ax = plt.subplots()
# ax.scatter(cdata['time'], cdata.humidity_bias_estimate, label='CDS')
# ax.scatter(adata['time'], adata.hur_bias_estimate, label='Calc')
# print(2)

# # ax.set(xlabel='time', ylabel='Adjustment: '+adj,
# #        title= str(station)+' Temperature '+str(plev)+' hPa '+time)
# # ax.legend()
# ax.grid()

# if save:
#     fig.savefig(str(station)+'_Temperature_'+str(plev)+'_hPa_'+time+'.png')
# plt.show()
# return